# Data Loading and Cleaning

In [4]:
# imports
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [5]:
# Loading csv with preprocessed data
# df = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data.csv')
df = pd.read_csv(r'C:\Users\Àiax\Desktop\BookRecommendationSystem\Preprocessed_data.csv')

# Cleaning the data
# Removing zero ratings
df = df[df['rating'] != 0]

In [6]:
# Removing image_url columns because we will not use it.
df = df.drop(['img_s'], axis=1)
df = df.drop(['img_m'], axis=1)
df = df.drop(['img_l'], axis=1)
df = df.dropna()

In [7]:
df.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
5,5,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],toronto,ontario,canada
9,9,123629,"kingston, ontario, canada",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],kingston,ontario,canada
11,11,200273,"comber, ontario, canada",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],comber,ontario,canada
12,12,210926,"guelph, ontario, canada",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],guelph,ontario,canada


In [8]:
df.to_csv('Preprocessed_data_cleaned.csv', index=False)

In [9]:
# From now on we can work with df or load from the beggining 
# the Preprocessed_data_cleaned.csv file

-----------------------------------------------------------
# Content Filtering - Variable based

In [64]:
# imports
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [65]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv(r'C:\Users\Àiax\Desktop\BookRecommendationSystem\Preprocessed_data_cleaned.csv')

print(df_cleaned)

        Unnamed: 0  user_id                   location      age        isbn  \
0                1        8   timmins, ontario, canada  34.7439  0002005018   
1                5    67544   toronto, ontario, canada  30.0000  0002005018   
2                9   123629  kingston, ontario, canada  34.7439  0002005018   
3               11   200273    comber, ontario, canada  34.7439  0002005018   
4               12   210926    guelph, ontario, canada  34.7439  0002005018   
...            ...      ...                        ...      ...         ...   
359466     1031169   278851         dallas, texas, usa  33.0000  067161746X   
359467     1031171   278851         dallas, texas, usa  33.0000  0767907566   
359468     1031172   278851         dallas, texas, usa  33.0000  0884159221   
359469     1031173   278851         dallas, texas, usa  33.0000  0912333022   
359470     1031174   278851         dallas, texas, usa  33.0000  1569661057   

        rating                                     

### book_title, book_author, publisher and Category recommender

In [66]:
# Data analysis
features = ['book_title', 'book_author', 'publisher', 'Category']
analysis_filename = 'data_analysis.txt'

for current_feature in features:
    unique_values = df_cleaned[current_feature].unique().tolist()

    with open(f"{analysis_filename}_{current_feature}", 'w', encoding="utf-8") as f:
        for v in unique_values:
            f.write("%s\n" % v)

# See how many times the category "9" appears
print(df_cleaned[features[3]].value_counts())


# Current features head
df_cleaned[features].head

9                                                         142157
['Fiction']                                               127055
['Juvenile Fiction']                                       14181
['Biography & Autobiography']                               8876
['Humor']                                                   3721
                                                           ...  
['Art patronage']                                              1
['Ethnobotany']                                                1
['Art criticism']                                              1
['Diners (Restaurants)']                                       1
['Authors, Canadian (English) 20th century Biography']         1
Name: Category, Length: 4162, dtype: int64


<bound method NDFrame.head of                                                book_title  \
0                                            Clara Callan   
1                                            Clara Callan   
2                                            Clara Callan   
3                                            Clara Callan   
4                                            Clara Callan   
...                                                   ...   
359466  The Bachelor Home Companion: A Practical Guide...   
359467  All Elevations Unknown: An Adventure in the He...   
359468  Why stop?: A guide to Texas historical roadsid...   
359469  The Are You Being Served? Stories: 'Camping In...   
359470  Dallas Street Map Guide and Directory, 2000 Ed...   

                 book_author                 publisher       Category  
0       Richard Bruce Wright     HarperFlamingo Canada  ['Actresses']  
1       Richard Bruce Wright     HarperFlamingo Canada  ['Actresses']  
2       Richard Bruce

### Drop duplicates

In [67]:
df_cleaned = df_cleaned.drop_duplicates(subset='isbn', keep="last")

# Current features head
df_cleaned[features].head

<bound method NDFrame.head of                                                book_title  \
6                                            Clara Callan   
8                                    Decision in Normandy   
14      Flu: The Story of the Great Influenza Pandemic...   
28                                 The Kitchen God's Wife   
29      What If?: The World's Foremost Military Histor...   
...                                                   ...   
359466  The Bachelor Home Companion: A Practical Guide...   
359467  All Elevations Unknown: An Adventure in the He...   
359468  Why stop?: A guide to Texas historical roadsid...   
359469  The Are You Being Served? Stories: 'Camping In...   
359470  Dallas Street Map Guide and Directory, 2000 Ed...   

                 book_author                 publisher       Category  
6       Richard Bruce Wright     HarperFlamingo Canada  ['Actresses']  
8               Carlo D'Este           HarperPerennial  ['1940-1949']  
14          Gina Bari

Clean category column

In [68]:
from ast import literal_eval

features = ['book_title', 'book_author', 'publisher', 'Category']

# Parse the stringified features into their corresponding python objects
df_cleaned[features[3]] = df_cleaned[features[3]].apply(literal_eval)

# Since each list contains only 1 element, replace the list by the element inside while dealing
# with the 9 values
def get_str_from_list(val):
    if val == 9:
        return str(9)
    return val[0]

df_cleaned[features[3]] = df_cleaned[features[3]].apply(get_str_from_list)

# Current features head
df_cleaned[features].head

<bound method NDFrame.head of                                                book_title  \
6                                            Clara Callan   
8                                    Decision in Normandy   
14      Flu: The Story of the Great Influenza Pandemic...   
28                                 The Kitchen God's Wife   
29      What If?: The World's Foremost Military Histor...   
...                                                   ...   
359466  The Bachelor Home Companion: A Practical Guide...   
359467  All Elevations Unknown: An Adventure in the He...   
359468  Why stop?: A guide to Texas historical roadsid...   
359469  The Are You Being Served? Stories: 'Camping In...   
359470  Dallas Street Map Guide and Directory, 2000 Ed...   

                 book_author                 publisher   Category  
6       Richard Bruce Wright     HarperFlamingo Canada  Actresses  
8               Carlo D'Este           HarperPerennial  1940-1949  
14          Gina Bari Kolata     

Prepare data

In [69]:
# Function to convert all strings to lower case and strip names of spaces
def lower_and_remove_spaces(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if author exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Apply clean_data function to selected features.
remove_spaces_features = ['book_author', 'publisher']

for feature in remove_spaces_features:
    df_cleaned[feature] = df_cleaned[feature].apply(lower_and_remove_spaces)

for feature in features:
    df_cleaned[feature] = df_cleaned[feature].apply(str.lower)


# Current features head
df_cleaned[features].head

<bound method NDFrame.head of                                                book_title         book_author  \
6                                            clara callan  richardbrucewright   
8                                    decision in normandy         carlod'este   
14      flu: the story of the great influenza pandemic...      ginabarikolata   
28                                 the kitchen god's wife              amytan   
29      what if?: the world's foremost military histor...        robertcowley   
...                                                   ...                 ...   
359466  the bachelor home companion: a practical guide...        p.j.o'rourke   
359467  all elevations unknown: an adventure in the he...         samlightner   
359468  why stop?: a guide to texas historical roadsid...        claudedooley   
359469  the are you being served? stories: 'camping in...         jeremylloyd   
359470  dallas street map guide and directory, 2000 ed...              mapsco  

Start recommender system

In [70]:
features = ['book_title', 'book_author', 'publisher', 'Category']

def create_soup(x):
    return ' '.join(x[features[0]]) + ' ' + ' '.join(x[features[1]]) + ' ' + x[features[2]] + ' ' + ' '.join(x[features[3]])
df_cleaned['soup'] = df_cleaned.apply(create_soup, axis=1)

print(df_cleaned['soup'].head)

<bound method NDFrame.head of 6         c l a r a   c a l l a n r i c h a r d b r u c ...
8         d e c i s i o n   i n   n o r m a n d y c a r ...
14        f l u :   t h e   s t o r y   o f   t h e   g ...
28        t h e   k i t c h e n   g o d ' s   w i f e a ...
29        w h a t   i f ? :   t h e   w o r l d ' s   f ...
                                ...                        
359466    t h e   b a c h e l o r   h o m e   c o m p a ...
359467    a l l   e l e v a t i o n s   u n k n o w n : ...
359468    w h y   s t o p ? :   a   g u i d e   t o   t ...
359469    t h e   a r e   y o u   b e i n g   s e r v e ...
359470    d a l l a s   s t r e e t   m a p   g u i d e ...
Name: soup, Length: 143883, dtype: object>


In [71]:
#df_cleaned = backup_df.copy()
backup_df = df_cleaned.copy()
df_cleaned = df_cleaned.head(20000)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Import CountVectorizer and create the count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_cleaned['soup'])

print(count_matrix.shape)

# Compute the Cosine Similarity matrix based on the count_matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Reset index of our main DataFrame and construct reverse mapping
df_cleaned = df_cleaned.reset_index()
indices = pd.Series(df_cleaned.index, index=df_cleaned['book_title'])

(20000, 2337)


In [72]:
print(indices)

book_title
clara callan                                                                                              0
decision in normandy                                                                                      1
flu: the story of the great influenza pandemic of 1918 and the search for the virus that caused it        2
the kitchen god's wife                                                                                    3
what if?: the world's foremost military historians imagine what might have been                           4
                                                                                                      ...  
the piper's sons                                                                                      19995
those who trespass: a novel of murder and television                                                  19996
the stargazey: a richard jury mystery (richard jury mysteries (paperback))                            19997
messiah          

In [73]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the book that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all book with that book
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1].any(), reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df_cleaned['book_title'].iloc[book_indices]

In [77]:
print(df_cleaned['book_title'].head)

print("\n\n\nRECOMMENDATIONS:")
get_recommendations('decision in normandy', cosine_sim)


<bound method NDFrame.head of 0                                             clara callan
1                                     decision in normandy
2        flu: the story of the great influenza pandemic...
3                                   the kitchen god's wife
4        what if?: the world's foremost military histor...
                               ...                        
19995                                     the piper's sons
19996    those who trespass: a novel of murder and tele...
19997    the stargazey: a richard jury mystery (richard...
19998                                              messiah
19999                               long, lean, and lethal
Name: book_title, Length: 20000, dtype: object>



RECOMMENDATIONS:


12840                                           lost girls
13289         tales of the city (tales of the city series)
0                                             clara callan
2        flu: the story of the great influenza pandemic...
3                                   the kitchen god's wife
4        what if?: the world's foremost military histor...
5                  where you'll find me: and other stories
6                              nights below station street
7                                       the middle stories
8                                                 jane doe
Name: book_title, dtype: object